# Model Evaluation

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from functions import get_current_weekday, calculate_nfl_week, get_next_sunday, get_current_year

In [11]:
import os
import tabulate

In [12]:
day = get_current_weekday()

In [13]:
date_string = get_next_sunday(day)

In [14]:
week = calculate_nfl_week(date_string)

In [15]:
season = get_current_year()

# RMSE testing scores
## Defense
**Ridge Regression:** 6.0157<br>
**Gradient Boost:** 6.0251<br>
**XGBoost:** 6.0249<br>

## Quarterback
### FanDuel
**Random Forest:** 6.7946<br>
**Gradient Boost:** 6.8371<br>
**XGBoost:** 6.826<br>
### DraftKings
**Random Forest:** 7.4053<br>
**Gradient Boost:** 7.4412<br>
**XGBoost:** 7.4497<br>

## FLEX
### FanDuel
**Random Forest:** 5.6812<br>
**Gradient Boost:** 5.6761<br>
**XGBoost:** 5.664<br>
### DraftKings
**Random Forest:** 6.7096<br>
**Gradient Boost:** 6.6893<br>
**XGBoost:** 6.6718<br>

In [16]:
#FD and DK predictions previous week
fd_preds = pd.read_csv('FD_predictions_' + str(season) + '_' + str(week - 1) + '.csv')
dk_preds = pd.read_csv('DK_predictions_' + str(season) + '_' + str(week - 1) + '.csv')

In [17]:
dk_preds.head()

,name,team,position,salary,opponent,status,season,week,pred_Ridge,pred_RF,pred_GB,pred_XGB,max_pred,min_pred,median_pred,value
0,Lamar Jackson,BAL,QB,7800,WAS,Active,2024.0,6,NaN,24.593434,25.460137,25.117788,25.460137,24.593434,25.117788,3.220229
1,Jayden Daniels,WAS,QB,7300,BAL,Active,2024.0,6,NaN,21.477548,21.390792,21.776040,21.776039,21.390792,21.477548,2.942130
2,Jared Goff,DET,QB,6400,DAL,Active,2024.0,6,NaN,21.294025,21.208996,21.044231,21.294025,21.044231,21.208996,3.313906
3,Jalen Hurts,PHI,QB,7200,CLE,Active,2024.0,6,NaN,20.181217,20.800140,20.618696,20.800140,20.181217,20.618696,2.863708
4,Dak Prescott,DAL,QB,6700,DET,Active,2024.0,6,NaN,20.349655,20.432040,20.687336,20.687336,20.349655,20.432040,3.049558


In [18]:
#Ground truth fantasy points for players previous week
def_target = pd.read_csv('def_target_' + str(season) + '_' + str(week - 1) + '.csv')
qb_target = pd.read_csv('qb_target_' + str(season) + '_' + str(week - 1) + '.csv')
flex_target = pd.read_csv('flex_target_' + str(season) + '_' + str(week - 1) + '.csv')

In [19]:
def_target.head()

,Unnamed: 0,season,week,game_id,team,fantasy_points,opponent
0,1294,2024,6,2024_06_ARI_GB,ARI,1.0,GB
1,1295,2024,6,2024_06_ARI_GB,GB,10.0,ARI
2,1296,2024,6,2024_06_ATL_CAR,ATL,5.0,CAR
3,1297,2024,6,2024_06_ATL_CAR,CAR,-2.0,ATL
4,1298,2024,6,2024_06_BUF_NYJ,BUF,6.0,NYJ


In [20]:
qb_target.head()

,Unnamed: 0,season,week,player_display_name,recent_team,opponent_team,position,FD_Pts,DK_Pts
0,1388,2024,6,Aaron Rodgers,NYJ,BUF,QB,18.76,18.76
1,1391,2024,6,Joe Flacco,IND,TEN,QB,14.66,14.66
2,1402,2024,6,Andy Dalton,CAR,ATL,QB,16.94,16.94
3,1409,2024,6,Kirk Cousins,ATL,CAR,QB,12.50,12.50
4,1415,2024,6,Geno Smith,SEA,SF,QB,14.48,17.48


In [21]:
flex_target.head()

,Unnamed: 0,season,week,player_display_name,recent_team,opponent_team,position,FD_Pts,DK_Pts
0,11117,2024,6,James Conner,ARI,GB,RB,4.6,7.6
1,11118,2024,6,Greg Dortch,ARI,GB,WR,3.1,5.6
2,11119,2024,6,Trey McBride,ARI,GB,TE,13.6,17.6
3,11120,2024,6,Michael Wilson,ARI,GB,WR,9.1,10.1
4,11121,2024,6,Emari Demercado,ARI,GB,RB,8.9,11.4


In [22]:
# season, week, team, opponent, name, position, FD_Pts, DK_Pts

In [23]:
fd_preds.drop(columns = ['status'], inplace = True)
dk_preds.drop(columns = ['status'], inplace = True)

In [24]:
def_target.drop(columns = ['Unnamed: 0', 'game_id'], inplace = True)
qb_target.drop(columns = ['Unnamed: 0'], inplace = True)
flex_target.drop(columns = ['Unnamed: 0'], inplace = True)

In [25]:
def_target['name'] = def_target['team']
def_target.rename(columns = {'fantasy_points': 'FD_Pts'}, inplace = True)
def_target['DK_Pts'] = def_target['FD_Pts']
def_target['position'] = 'D'

In [26]:
qb_target.rename(columns = {'player_display_name': 'name', 'recent_team': 'team', 'opponent_team': 'opponent'}, inplace = True)
flex_target.rename(columns = {'player_display_name': 'name', 'recent_team': 'team', 'opponent_team': 'opponent'}, inplace = True)

In [27]:
def_target.head()

,season,week,team,FD_Pts,opponent,name,DK_Pts,position
0,2024,6,ARI,1.0,GB,ARI,1.0,D
1,2024,6,GB,10.0,ARI,GB,10.0,D
2,2024,6,ATL,5.0,CAR,ATL,5.0,D
3,2024,6,CAR,-2.0,ATL,CAR,-2.0,D
4,2024,6,BUF,6.0,NYJ,BUF,6.0,D


In [28]:
qb_target.head(10)

,season,week,name,team,opponent,position,FD_Pts,DK_Pts
0,2024,6,Aaron Rodgers,NYJ,BUF,QB,18.76,18.76
1,2024,6,Joe Flacco,IND,TEN,QB,14.66,14.66
2,2024,6,Andy Dalton,CAR,ATL,QB,16.94,16.94
3,2024,6,Kirk Cousins,ATL,CAR,QB,12.50,12.50
4,2024,6,Geno Smith,SEA,SF,QB,14.48,17.48
5,2024,6,Dak Prescott,DAL,DET,QB,5.22,5.22
6,2024,6,Jared Goff,DET,DAL,QB,25.10,28.10
7,2024,6,Deshaun Watson,CLE,PHI,QB,8.42,8.42
8,2024,6,Cooper Rush,DAL,DET,QB,1.04,1.04
9,2024,6,Lamar Jackson,BAL,WAS,QB,19.92,22.92


In [29]:
flex_target.head()

,season,week,name,team,opponent,position,FD_Pts,DK_Pts
0,2024,6,James Conner,ARI,GB,RB,4.6,7.6
1,2024,6,Greg Dortch,ARI,GB,WR,3.1,5.6
2,2024,6,Trey McBride,ARI,GB,TE,13.6,17.6
3,2024,6,Michael Wilson,ARI,GB,WR,9.1,10.1
4,2024,6,Emari Demercado,ARI,GB,RB,8.9,11.4


In [30]:
all_target = pd.concat([def_target, qb_target, flex_target], ignore_index = True)

In [31]:
fd_target = all_target.drop(columns = ['DK_Pts'])
dk_target = all_target.drop(columns = ['FD_Pts'])

In [32]:
fd_eval = pd.merge(fd_preds, fd_target, on = ['season', 'week', 'name', 'team', 'opponent', 'position'], how = 'left')

In [33]:
dk_eval = pd.merge(dk_preds, dk_target, on = ['season', 'week', 'name', 'team', 'opponent', 'position'], how = 'left')

In [34]:
fd_eval.head()

,Unnamed: 0,name,team,position,salary,opponent,season,week,pred_Ridge,pred_RF,pred_GB,pred_XGB,max_pred,min_pred,median_pred,value,FD_Pts
0,0,Lamar Jackson,BAL,QB,9500,WAS,2024,6,NaN,23.445014,22.836345,24.080180,24.080179,22.836345,23.445014,2.467896,19.92
1,1,Jayden Daniels,WAS,QB,8700,BAL,2024,6,NaN,21.126638,20.762438,21.881880,21.881880,20.762438,21.126638,2.428349,20.96
2,2,Jalen Hurts,PHI,QB,8300,CLE,2024,6,NaN,20.120971,19.809217,19.955610,20.120971,19.809217,19.955610,2.404290,21.86
3,3,Jared Goff,DET,QB,7700,DAL,2024,6,NaN,19.771264,19.508634,19.469610,19.771264,19.469610,19.508634,2.533589,25.10
4,4,Joe Burrow,CIN,QB,8100,NYG,2024,6,NaN,18.649665,19.112493,18.917742,19.112493,18.649665,18.917742,2.335524,19.82


In [35]:
fd_eval.drop(columns = ['max_pred', 'min_pred', 'median_pred'], inplace = True)
dk_eval.drop(columns = ['max_pred', 'min_pred', 'median_pred'], inplace = True)

In [36]:
fd_eval.dropna(subset = ['FD_Pts'], inplace = True)
dk_eval.dropna(subset = ['DK_Pts'], inplace = True)
# fd_eval['FD_Pts'].fillna(0, inplace=True)
# dk_eval['DK_Pts'].fillna(0, inplace=True)


In [37]:
fd_eval.info()

<class 'pandas.core.frame.DataFrame'>
Index: 184 entries, 0 to 262
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  184 non-null    int64  
 1   name        184 non-null    object 
 2   team        184 non-null    object 
 3   position    184 non-null    object 
 4   salary      184 non-null    int64  
 5   opponent    184 non-null    object 
 6   season      184 non-null    int64  
 7   week        184 non-null    int64  
 8   pred_Ridge  22 non-null     float64
 9   pred_RF     162 non-null    float64
 10  pred_GB     184 non-null    float64
 11  pred_XGB    184 non-null    float64
 12  value       184 non-null    float64
 13  FD_Pts      184 non-null    float64
dtypes: float64(6), int64(4), object(4)
memory usage: 21.6+ KB


In [38]:
dk_eval.info()

<class 'pandas.core.frame.DataFrame'>
Index: 208 entries, 0 to 431
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        208 non-null    object 
 1   team        208 non-null    object 
 2   position    208 non-null    object 
 3   salary      208 non-null    int64  
 4   opponent    208 non-null    object 
 5   season      208 non-null    float64
 6   week        208 non-null    int64  
 7   pred_Ridge  20 non-null     float64
 8   pred_RF     188 non-null    float64
 9   pred_GB     208 non-null    float64
 10  pred_XGB    208 non-null    float64
 11  value       208 non-null    float64
 12  DK_Pts      208 non-null    float64
dtypes: float64(7), int64(2), object(4)
memory usage: 22.8+ KB


In [39]:
len(fd_eval), len(dk_eval)

(184, 208)

In [40]:
fd_eval_def = fd_eval[fd_eval['position'] == 'D']
fd_eval_qb = fd_eval[fd_eval['position'] == 'QB']
fd_eval_flex = fd_eval[fd_eval['position'].isin(['RB', 'WR', 'TE'])]
dk_eval_def = dk_eval[dk_eval['position'] == 'D']
dk_eval_qb = dk_eval[dk_eval['position'] == 'QB']
dk_eval_flex = dk_eval[dk_eval['position'].isin(['RB', 'WR', 'TE'])]

In [41]:
len(fd_eval_def) + len(fd_eval_qb) + len(fd_eval_flex) == len(fd_eval)

True

In [42]:
len(dk_eval_def) + len(dk_eval_qb) + len(dk_eval_flex) == len(dk_eval)

True

In [43]:
dk_eval_def.head()

,name,team,position,salary,opponent,season,week,pred_Ridge,pred_RF,pred_GB,pred_XGB,value,DK_Pts
65,HOU,HOU,D,3600,NE,2024.0,6,10.781708,NaN,10.765499,10.779221,2.994228,12.0
67,PHI,PHI,D,3400,CLE,2024.0,6,10.538316,NaN,10.630450,10.636938,3.126603,9.0
77,PIT,PIT,D,3700,LV,2024.0,6,9.815382,NaN,9.754449,9.778424,2.642817,13.0
80,LAC,LAC,D,3300,DEN,2024.0,6,9.963380,NaN,9.678152,9.694053,2.937592,7.0
99,TB,TB,D,2600,NO,2024.0,6,8.562224,NaN,8.916703,8.881475,3.415952,17.0


In [44]:
rmse_def_dict = {'RMSE_Ridge': 'pred_Ridge', 'RMSE_GB': 'pred_GB', 'RMSE_XGB': 'pred_XGB'}
rmse_dict = {'RMSE_RF': 'pred_RF', 'RMSE_GB': 'pred_GB', 'RMSE_XGB': 'pred_XGB'}

In [45]:
fd_rmse_def = {}
fd_rmse_qb = {}
fd_rmse_flex = {}
dk_rmse_def = {}
dk_rmse_qb = {}
dk_rmse_flex = {}

In [46]:
for k, v in rmse_def_dict.items():
    fd_rmse_def[k] = np.sqrt(((fd_eval_def[v] - fd_eval_def['FD_Pts']) ** 2).mean())
    

In [47]:
for k, v in rmse_def_dict.items():
    dk_rmse_def[k] = np.sqrt(((dk_eval_def[v] - dk_eval_def['DK_Pts']) ** 2).mean())    

In [48]:
for k, v in rmse_dict.items():
    fd_rmse_qb[k] = np.sqrt(((fd_eval_qb[v] - fd_eval_qb['FD_Pts']) ** 2).mean())
    fd_rmse_flex[k] = np.sqrt(((fd_eval_flex[v] - fd_eval_flex['FD_Pts']) ** 2).mean())
    dk_rmse_qb[k] = np.sqrt(((dk_eval_qb[v] - dk_eval_qb['DK_Pts']) ** 2).mean())
    dk_rmse_flex[k] = np.sqrt(((dk_eval_flex[v] - dk_eval_flex['DK_Pts']) ** 2).mean())

In [49]:
fd_rmse_def

{'RMSE_Ridge': 4.940156840292998,
 'RMSE_GB': 5.0588669646235775,
 'RMSE_XGB': 5.060821851762603}

In [50]:
dk_rmse_def

{'RMSE_Ridge': 5.164205823036328,
 'RMSE_GB': 5.29610974781583,
 'RMSE_XGB': 5.295816316574046}

In [51]:
fd_rmse_qb

{'RMSE_RF': 6.306949112050349,
 'RMSE_GB': 6.103322581243178,
 'RMSE_XGB': 6.02755811495002}

In [52]:
dk_rmse_qb

{'RMSE_RF': 6.812723583760086,
 'RMSE_GB': 6.647558627173642,
 'RMSE_XGB': 6.606350868259097}

In [53]:
fd_rmse_flex

{'RMSE_RF': 5.504295731123421,
 'RMSE_GB': 5.504432471460074,
 'RMSE_XGB': 5.495269693868216}

In [54]:
dk_rmse_flex

{'RMSE_RF': 6.547126737481695,
 'RMSE_GB': 6.41776419326847,
 'RMSE_XGB': 6.434427978951407}

In [55]:
error_dict_for_df = {'FD_Ridge_D': fd_rmse_def['RMSE_Ridge'], 'FD_GB_D': fd_rmse_def['RMSE_GB'], 'FD_XGB_D': fd_rmse_def['RMSE_XGB'],\
                    'DK_Ridge_D': dk_rmse_def['RMSE_Ridge'], 'DK_GB_D': dk_rmse_def['RMSE_GB'], 'DK_XGB_D': dk_rmse_def['RMSE_XGB'],\
                    'FD_RF_QB': fd_rmse_qb['RMSE_RF'], 'FD_GB_QB': fd_rmse_qb['RMSE_GB'], 'FD_XGB_QB': fd_rmse_qb['RMSE_XGB'],\
                    'DK_RF_QB': dk_rmse_qb['RMSE_RF'], 'DK_GB_QB': dk_rmse_qb['RMSE_GB'], 'DK_XGB_QB': dk_rmse_qb['RMSE_XGB'],\
                    'FD_RF_FLEX': fd_rmse_flex['RMSE_RF'], 'FD_GB_FLEX': fd_rmse_flex['RMSE_GB'], 'FD_XGB_FLEX': fd_rmse_flex['RMSE_XGB'],
                    'DK_RF_FLEX': dk_rmse_flex['RMSE_RF'], 'DK_GB_FLEX': dk_rmse_flex['RMSE_GB'], 'DK_XGB_FLEX': dk_rmse_flex['RMSE_XGB']}

In [58]:
error_df = pd.DataFrame(error_dict_for_df, index = [f"{str(season)} Week {str(week - 1)}"])

In [59]:
error_df

,FD_Ridge_D,FD_GB_D,FD_XGB_D,DK_Ridge_D,DK_GB_D,DK_XGB_D,FD_RF_QB,FD_GB_QB,FD_XGB_QB,DK_RF_QB,DK_GB_QB,DK_XGB_QB,FD_RF_FLEX,FD_GB_FLEX,FD_XGB_FLEX,DK_RF_FLEX,DK_GB_FLEX,DK_XGB_FLEX
2024 Week 6,4.940157,5.058867,5.060822,5.164206,5.29611,5.295816,6.306949,6.103323,6.027558,6.812724,6.647559,6.606351,5.504296,5.504432,5.49527,6.547127,6.417764,6.434428


In [60]:
error_df.to_csv(f"Model_Eval_{str(season)}_Week_{str(week - 1)}.csv")

In [76]:
main_error_df = pd.read_csv(f"Model_Eval_{str(season)}}.csv", index_col = 0)

In [77]:
main_error_df

,FD_Ridge_D,FD_GB_D,FD_XGB_D,DK_Ridge_D,DK_GB_D,DK_XGB_D,FD_RF_QB,FD_GB_QB,FD_XGB_QB,DK_RF_QB,DK_GB_QB,DK_XGB_QB,FD_RF_FLEX,FD_GB_FLEX,FD_XGB_FLEX,DK_RF_FLEX,DK_GB_FLEX,DK_XGB_FLEX
2024 Week 6,4.940157,5.058867,5.060822,5.164206,5.29611,5.295816,6.306949,6.103323,6.027558,6.812724,6.647559,6.606351,5.504296,5.504432,5.49527,6.547127,6.417764,6.434428


In [80]:
error_df = pd.concat([main_error_df, error_df], axis = 0)#error_df.T

In [81]:
error_df

,FD_Ridge_D,FD_GB_D,FD_XGB_D,DK_Ridge_D,DK_GB_D,DK_XGB_D,FD_RF_QB,FD_GB_QB,FD_XGB_QB,DK_RF_QB,DK_GB_QB,DK_XGB_QB,FD_RF_FLEX,FD_GB_FLEX,FD_XGB_FLEX,DK_RF_FLEX,DK_GB_FLEX,DK_XGB_FLEX
2024 Week 3,5.439323,5.442453,5.433941,5.560427,5.459133,5.444914,6.404433,6.329214,6.275229,6.887222,7.025536,6.907759,6.340756,6.299230,6.325915,7.690199,7.677433,7.646078
2024 Week 6,4.940157,5.058867,5.060822,5.164206,5.296110,5.295816,6.306949,6.103323,6.027558,6.812724,6.647559,6.606351,5.504296,5.504432,5.495270,6.547127,6.417764,6.434428


In [ ]:
error_df.to_csv(f"error_df_{str(season)}.csv")

In [73]:
error_df = error_df.T

In [75]:
# Export DataFrame to markdown
md_table = error_df.to_markdown()

# Save the markdown table to a file
with open('table_for_readme.md', 'w') as f:
    f.write(md_table)
